In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[10]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,9401.0,0.228165,1.869,0.0,0.0
1,8957.0,0.193447,1.625,0.0,0.0
2,8462.0,0.154742,1.448,0.0,0.0
3,8156.0,0.130816,1.250,0.0,0.0
4,8126.0,0.128470,1.102,0.0,0.0
...,...,...,...,...,...
43819,12799.0,0.493862,2.909,0.0,0.0
43820,12065.0,0.436469,2.635,0.0,0.0
43821,11641.0,0.403315,2.210,0.0,0.0
43822,11418.0,0.385878,1.785,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[10]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_NL,Temp_NL
0,9401.0,1.869
1,8957.0,1.625
2,8462.0,1.448
3,8156.0,1.250
4,8126.0,1.102
...,...,...
43819,12799.0,2.909
43820,12065.0,2.635
43821,11641.0,2.210
43822,11418.0,1.785


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_NL,Temp_NL,Lag-2,Lag-7
0,9341.0,2.153,12082.0,9401.0
1,9008.0,2.465,11748.0,8957.0
2,8889.0,2.813,11644.0,8462.0
3,8929.0,3.286,11709.0,8156.0
4,9424.0,3.831,12060.0,8126.0
...,...,...,...,...
43651,12799.0,2.909,13718.0,13265.0
43652,12065.0,2.635,13133.0,12680.0
43653,11641.0,2.210,12522.0,12073.0
43654,11418.0,1.785,11708.0,11622.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 14s 9ms/step - loss: 0.0166 - mape: 5208.9912 - mae: 0.0994 - val_loss: 0.0207 - val_mape: 19.8103 - val_mae: 0.1112
Epoch 2/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0078 - mape: 3678.7610 - mae: 0.0665 - val_loss: 0.0097 - val_mape: 15.8572 - val_mae: 0.0828
Epoch 3/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0064 - mape: 3667.1816 - mae: 0.0594 - val_loss: 0.0100 - val_mape: 16.4676 - val_mae: 0.0853
Epoch 4/100
1105/1105 [==============================] - 9s 9ms/step - loss: 0.0058 - mape: 3222.4895 - mae: 0.0555 - val_loss: 0.0120 - val_mape: 17.9814 - val_mae: 0.0941
Epoch 5/100
1105/1105 [==============================] - 9s 9ms/step - loss: 0.0053 - mape: 3591.6069 - mae: 0.0529 - val_loss: 0.0177 - val_mape: 23.1454 - val_mae: 0.1186
Epoch 6/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0051 - mape: 3123.0525 - mae: 0.0512 - val_loss: 0.0

1105/1105 [==============================] - 8s 7ms/step - loss: 0.0018 - mape: 1466.7747 - mae: 0.0306 - val_loss: 0.0108 - val_mape: 16.8899 - val_mae: 0.0837
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0018 - mape: 1286.4927 - mae: 0.0302 - val_loss: 0.0150 - val_mape: 20.0030 - val_mae: 0.0960
Epoch 50/100
1105/1105 [==============================] - 8s 8ms/step - loss: 0.0017 - mape: 1199.4346 - mae: 0.0297 - val_loss: 0.0122 - val_mape: 18.2032 - val_mae: 0.0879
Epoch 51/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0017 - mape: 1154.0533 - mae: 0.0295 - val_loss: 0.0108 - val_mape: 16.4485 - val_mae: 0.0836
Epoch 52/100
1105/1105 [==============================] - 10s 9ms/step - loss: 0.0017 - mape: 1361.7446 - mae: 0.0294 - val_loss: 0.0100 - val_mape: 15.6369 - val_mae: 0.0788
Epoch 53/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0017 - mape: 1245.3378 - mae: 0.0294 - val_loss: 0.0116 - val_

1105/1105 [==============================] - 9s 8ms/step - loss: 9.8788e-04 - mape: 1280.5593 - mae: 0.0231 - val_loss: 0.0121 - val_mape: 17.5512 - val_mae: 0.0896
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 9.3778e-04 - mape: 1067.7815 - mae: 0.0226 - val_loss: 0.0136 - val_mape: 18.3500 - val_mae: 0.0925
Epoch 97/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.6758e-04 - mape: 1073.8962 - mae: 0.0229 - val_loss: 0.0106 - val_mape: 16.7046 - val_mae: 0.0826
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0010 - mape: 1131.2615 - mae: 0.0235 - val_loss: 0.0121 - val_mape: 17.5140 - val_mae: 0.0892
Epoch 99/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.6268e-04 - mape: 1262.6841 - mae: 0.0228 - val_loss: 0.0101 - val_mape: 16.1185 - val_mae: 0.0815
Epoch 100/100
1105/1105 [==============================] - 8s 8ms/step - loss: 9.6122e-04 - mape: 1185.6859 - mae: 0.0228 - val

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 876.1113169193268


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.27162677, 0.2398089 , 0.24078214, ..., 0.40556067, 0.36857817,
       0.35180002], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.35744, 0.34448, 0.34344, ..., 0.41264, 0.3948 , 0.37856])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,10951.0,9878.333984
1,10789.0,9480.611328
2,10776.0,9492.776367
3,10842.0,9638.143555
4,11558.0,10062.678711
...,...,...
4363,12799.0,13260.385742
4364,12065.0,12342.008789
4365,11641.0,11552.507812
4366,11418.0,11090.227539


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

5.916817208852109